# Import des packages

In [12]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import requests


# Import des données

## Données des points d'apports volontaires

In [10]:
#lecture du fichier
url = "https://www.data.gouv.fr/api/1/datasets/r/e46c6879-49e7-4727-8f3b-62df90ac5a5a"
response = requests.get(url)
with open("../data/raw/aix_marseille_pav.csv", "wb") as file:
    file.write(response.content)
df_aix_marseille = pd.read_csv("../data/raw/aix_marseille_pav.csv", sep=";")

In [163]:
#On sélectionne uniquement les lignes correspondant à Marseille
df_marseille = df_aix_marseille[df_aix_marseille["Commune"].str.startswith('Marseille')]
df_marseille

,OBJECTID,Ex identifiant CT,Date création mobilier,Famille Osis,Type de colonne,Modèle,Volume intérieur m3,longitude,latitude,Date de mise à jour de la donnée,...,Code postal,Quartier,Accessibilité,Nom de colonne,Abréviation type de flux,Trigramme de la commune,Propriétaire,created_date,last_edited_date,geo_point_2d
43,126605,PAV07118,2022-12-31T01:00:00+00:00,Equipement Mobilier Urbain,Aérien,VAGUE,3.0,5.360057,43.281150,2026-01-22T15:01:37+00:00,...,13007.0,BOMPARD,NaN,NaN,NaN,MRS7,NaN,2025-12-22T15:36:07+00:00,2024-02-02T11:13:06+00:00,"43.281149769353924, 5.360056706275721"
44,126607,PAV02034,2022-12-31T01:00:00+00:00,Equipement Mobilier Urbain,Aérien,VAGUE,3.0,5.366560,43.299830,2025-12-22T15:36:07+00:00,...,13002.0,LES GRANDS CARMES,NaN,NaN,NaN,MRS2,NaN,2025-12-22T15:36:07+00:00,2024-02-02T10:16:19+00:00,"43.29983000682552, 5.366559561090769"
45,126617,PAV12030,2022-12-31T01:00:00+00:00,Equipement Mobilier Urbain,Aérien,COMPOECO METAL,3.0,5.416391,43.312216,2025-12-30T11:42:59+00:00,...,13012.0,SAINT BARNABE,NaN,NaN,NaN,MRS12,NaN,2025-12-22T15:36:07+00:00,2024-10-16T14:44:09+00:00,"43.312216006367564, 5.416390546482677"
46,126622,PAV12031,2022-12-31T01:00:00+00:00,Equipement Mobilier Urbain,Aérien,COMPOECO,3.0,5.419281,43.313656,2025-12-22T15:36:07+00:00,...,13012.0,MONTOLIVET,NaN,NaN,NaN,MRS12,Propriétaire public,2025-12-22T15:36:07+00:00,2024-02-02T10:52:54+00:00,"43.31365573263991, 5.419280605843906"
47,126624,PAV09174,2022-12-31T01:00:00+00:00,Equipement Mobilier Urbain,Aérien,VAGUE,3.0,5.399052,43.255417,2025-12-22T15:36:07+00:00,...,13009.0,MAZARGUES,NaN,NaN,NaN,MRS9,Propriétaire Privé,2025-12-22T15:36:07+00:00,2024-02-02T10:57:54+00:00,"43.25541720547701, 5.3990518110479595"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12768,136066,ENT09181,2022-12-31T01:00:00+00:00,Equipement Mobilier Urbain,Enterré,CITEC T5,3.0,5.402791,43.231340,2026-01-23T11:42:25+00:00,...,13009.0,SORMIOU,NaN,NaN,NaN,MRS,Propriétaire public,2025-12-22T15:36:07+00:00,2024-02-02T10:57:54+00:00,"43.23134046656937, 5.402790540955334"
12769,136067,NaN,2022-12-31T01:00:00+00:00,Equipement Mobilier Urbain,Aérien,AUTRE,NaN,5.397462,43.257937,2025-12-22T15:36:07+00:00,...,13009.0,NaN,NaN,NaN,NaN,MRS9,Propriétaire public,2025-12-22T15:36:07+00:00,2024-02-02T10:57:54+00:00,"43.257936999830534, 5.397461999816812"
12770,136069,NaN,2022-12-31T01:00:00+00:00,Equipement Mobilier Urbain,Aérien,AUTRE,NaN,5.411851,43.245889,2025-12-22T15:36:07+00:00,...,13009.0,NaN,NaN,NaN,NaN,MRS9,Propriétaire public,2025-12-22T15:36:07+00:00,2024-02-02T10:57:54+00:00,"43.245889000257066, 5.411851000377483"
12771,136076,NaN,2022-12-31T01:00:00+00:00,Equipement Mobilier Urbain,Aérien,AUTRE,NaN,5.420326,43.255083,2025-12-22T15:36:07+00:00,...,13009.0,NaN,NaN,NaN,NaN,MRS9,Propriétaire public,2025-12-22T15:36:07+00:00,2024-02-02T10:57:54+00:00,"43.255082999737695, 5.42032600024191"


In [ ]:
# Conversion du DataFrame en GeoDataFrame
gdf_marseille = gpd.GeoDataFrame(
    df_marseille, 
    geometry=gpd.points_from_xy(df_marseille.longitude, df_marseille.latitude),
    crs="EPSG:4326"
)

## Import des données géographiques des quartiers

In [14]:
#téléchargement du GeoJSON des quartiers de Marseille
url_quartiers = "https://www.data.gouv.fr/api/1/datasets/r/8a8f7f54-7f91-482c-a78c-dd09d893d1b6"
with open("../data/raw/marseille_quartiers.geojson", "wb") as file:
    file.write(requests.get(url_quartiers).content)
quartiers = gpd.read_file("../data/raw/marseille_quartiers.geojson")

In [ ]:
quartiers

In [15]:
#nettoyage du nom du quartier "LES ILES"
quartiers.loc[30, 'NOM_QUA'] = 'LES ILES'

In [167]:
#ajout des points sur la carte
for idx, row in gdf_marseille.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5,
        popup=f"Adresse: {row['Quartier']}\nCommune: {row['Commune']}\nCode Postal: {row['Code postal']}",
        color="blue",
        fill=True,
        fill_color="blue",
    ).add_to(Carte)

## Import des données de population

In [168]:
df_pop = pandas.read_csv("../data/raw/pop_marseille_2022.csv", sep=";")
df_pop

,quartier,population
0,Belsunce,8825
1,Chapitre,6497
2,Noailles,4304
3,Opéra,5468
4,Saint-Charles,8390
...,...,...
106,La Viste,6665
107,L'Estaque,5807
108,Les Riaux,722
109,Saint-André,3739


In [169]:
#supprimer accents et tirets de df_pop['quartier']
df_pop['quartier_upper_clean'] = (df_pop['quartier']
                                  .str.upper()
                                  .str.normalize('NFKD')
                                  .str.encode('ascii', errors='ignore')
                                  .str.decode('utf-8')
                                  .str.replace('-',' ')
)
df_pop

,quartier,population,quartier_upper_clean
0,Belsunce,8825,BELSUNCE
1,Chapitre,6497,CHAPITRE
2,Noailles,4304,NOAILLES
3,Opéra,5468,OPERA
4,Saint-Charles,8390,SAINT CHARLES
...,...,...,...
106,La Viste,6665,LA VISTE
107,L'Estaque,5807,L'ESTAQUE
108,Les Riaux,722,LES RIAUX
109,Saint-André,3739,SAINT ANDRE


# Jointure des df de population et des quartiers

In [170]:
# On va modifier certains noms de quartiers pour qu'ils correspondent entre les deux DataFrames
mapping = {'SAINT MAURON': 'SAINT MAURONT',
           'LA VILETTE': 'VILLETTE',
           'LA BLANCARDE': 'BLANCARDE',
           'LES CHARTREUX': 'CHARTREUX',
           'LE CAMAS': 'CAMAS',
            'LA CONCEPTION': 'CONCEPTION',
            'LE PHARO': 'PHARO',
            'LE ROUET': 'ROUET',
            'VIELLE CHAPELLE': 'VIEILLE CHAPELLE',
            'LES BAUMETTES': 'BAUMETTES',
            'LE REDON': 'REDON',
            'LA CAPELETTE': 'CAPELETTE',
            'CHATEAU-GOMBERT': 'CHATEAU GOMBERT',
            'LA CROIX ROUGE': 'CROIX ROUGE',
            'LES MOURETS': 'MOURETS',
            'LES BORELS': 'BORELS',
            }

In [172]:
df_pop['NOM_QUA'] = quartiers['NOM_QUA']

In [173]:
df_pop['NOM_QUA'] = df_pop['NOM_QUA'].replace(mapping)
quartiers['NOM_QUA'] = quartiers['NOM_QUA'].replace(mapping)

In [174]:
df_pop_quartiers = df_pop.merge(quartiers, on='NOM_QUA')

In [175]:
df_pop_quartiers

,quartier,population,quartier_upper_clean,NOM_QUA,DEPCO,NOM_CO,geometry
0,Belsunce,8825,BELSUNCE,BELSUNCE,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.38086 43.29924, 5.38087 43.2..."
1,Chapitre,6497,CHAPITRE,CHAPITRE,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.38525 43.29906, 5.38485 43.2..."
2,Noailles,4304,NOAILLES,NOAILLES,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.3816 43.29573, 5.38177 43.29..."
3,Opéra,5468,OPERA,OPERA,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.37729 43.29222, 5.37663 43.2..."
4,Saint-Charles,8390,SAINT CHARLES,SAINT CHARLES,13201,Marseille 1er Arrondissemen,"MULTIPOLYGON (((5.38022 43.30141, 5.38007 43.3..."
...,...,...,...,...,...,...,...
106,La Viste,6665,LA VISTE,LA VISTE,13215,Marseille 15e Arrondissemen,"MULTIPOLYGON (((5.35771 43.35337, 5.35756 43.3..."
107,L'Estaque,5807,L'ESTAQUE,L'ESTAQUE,13216,Marseille 16e Arrondissemen,"MULTIPOLYGON (((5.32691 43.36112, 5.32694 43.3..."
108,Les Riaux,722,LES RIAUX,LES RIAUX,13216,Marseille 16e Arrondissemen,"MULTIPOLYGON (((5.30862 43.36508, 5.30878 43.3..."
109,Saint-André,3739,SAINT ANDRE,SAINT ANDRE,13216,Marseille 16e Arrondissemen,"MULTIPOLYGON (((5.33997 43.34367, 5.33992 43.3..."


# Comptage du nombre de pav à l'intérieur des quartiers

In [176]:
#convertir en GeoDataFrame
df_pop_quartiers = geopandas.GeoDataFrame(df_pop_quartiers, geometry='geometry')

In [177]:
#nombre de coordonnées à l'intérieur de chaque quartier
gdf_marseille = gdf_marseille.to_crs(df_pop_quartiers.crs)
join = gpd.sjoin(gdf_marseille, df_pop_quartiers, how="inner", predicate='within')
count = join.groupby('NOM_QUA').size().reset_index(name='pav_count')
df_pop_quartiers['nb_pav'] = count['pav_count']

## Création de l'indicateur voulu

In [178]:
df_pop_quartiers['pav_per_1000'] = (df_pop_quartiers['nb_pav'] / df_pop_quartiers['population']) * 1000

In [186]:
df_pop_quartiers['pav_per_1000'] = df_pop_quartiers['pav_per_1000'].round(2)

In [204]:
df_pop_quartiers['pav_per_1000'].sort_values(ascending=False)

30     754.46
47      93.33
53      83.61
81      65.70
36      59.12
        ...  
62       0.74
37       0.55
54       0.53
52       0.19
110       NaN
Name: pav_per_1000, Length: 111, dtype: float64

In [223]:
outlier = df_pop_quartiers.loc[
    df_pop_quartiers["pav_per_1000"] >= 754
]

outlier[["NOM_QUA", "pav_per_1000"]]

,NOM_QUA,pav_per_1000
30,LES ILES,754.46


# Création de la carte

In [230]:
# Créer la carte
location = [43.2962184326563, 5.369918462070215]  # mairie de Marseille
zoom = 13
Carte = folium.Map(location=location, zoom_start=zoom, tiles="cartodbpositron")

In [231]:
gdf_main = df_pop_quartiers[df_pop_quartiers["pav_per_1000"] < 754]

bins = [0, 10, 20, 40, 60, 100]

folium.Choropleth(
    geo_data=gdf_main,
    data=gdf_main,
    columns=["NOM_QUA", "pav_per_1000"],
    key_on="feature.properties.NOM_QUA",
    fill_color="YlGn",
    bins=bins,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="PAV pour 1000 habitants (hors outlier)"
).add_to(Carte)


In [232]:
folium.GeoJson(
    outlier,
    style_function=lambda x: {
        "fillColor": "#ff0000",
        "color": "#000000",
        "weight": 2,
        "fillOpacity": 0.9,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NOM_QUA", "pav_per_1000"],
        aliases=["Quartier", "PAV / 1000 hab"]
    )
).add_to(Carte)

In [233]:
#on rajoute les quartiers à la carte et on affiche le nom des quartiers survolés
folium.GeoJson(
    df_pop_quartiers,
    style_function=lambda feature: {
        "fillColor": "transparent",   # pas de remplissage
        "color": "#000000",           # couleur des contours
        "weight": 0.3,                # traits très fins
        "fillOpacity": 0.0,           # transparence totale
        "opacity": 0.3                # contours légers
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NOM_QUA", "pav_per_1000"],
        aliases=["Quartier", "PAV / 1000 hab"],
        localize=True
    )
).add_to(Carte)

In [234]:
Carte.save("../output/pav_marseille.html")